In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

# USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


# Breast Cancer Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [3]:
my_random_seed = 42
random.seed(my_random_seed)

In [4]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [5]:
def load_dataset(fp, 
                 randomSeed = 42, 
                 proportionValidation = 0.2):
    ''' Takes in a filepath, returns x_train, x_validate, y_train, y_validate'''
    df = pd.read_csv(fp).reset_index(drop=True)
    xs_raw = df[[
        'ag_age',
        'ag_sex',
        'ag_eth',
        'pt_nzdep',
        'imp_hxdiab',
        'pt_tc_hdl_ratio',
        'pt_bps',
        'pt_bpd',
        'pt_smoke',
        'imp_hxcvd',
        'imp_hdl',
        'imp_ldl',
        'imp_tchol',
        'marker',
        'region',
        'PH_BL_LLD_ANY',
        'PH_BL_AHT_ANY',
        'pt_familyhistory',
        'ab_gen',
        'eth_gen',
        'is.female',
        'log.age',
        'log.age.gender',
        'log.sbp',
        'smoking',
        'log.tchdl',
        'diabetes',
        'diabetes.sex']]
    
    xs_raw = xs_raw[[
        'is.female', 
        'ag_age',
        'pt_bps',
        'smoking',
        'pt_tc_hdl_ratio',
        'diabetes'
    ]]


    scaler = StandardScaler()
    
    scaler.fit(xs_raw)
    xs = scaler.transform(xs_raw)
    ys = df['dead'].apply(lambda x: 1 if x else 0)
    ys = np.array(ys).astype(float)
    if proportionValidation == 0:
        return xs, [], ys, []
    X_train, X_validate, y_train, y_validate = train_test_split(xs, ys, test_size=proportionValidation, random_state=randomSeed)
    return X_train, X_validate, y_train, y_validate
    


In [6]:
X_test, _, y_test, __ = load_dataset('./../../data/processed/synthetic_view/synthetic_view_test.csv',
                                    proportionValidation = 0)

In [7]:
X_test.shape

(451278, 6)

In [8]:
X_test[:5]

array([[ 0.95547078, -1.2099392 , -0.95424448, -0.86593193, -0.61149249,
        -0.3018414 ],
       [-1.04660448,  0.24162668,  0.25840041,  1.15482518, -1.58208848,
        -0.3018414 ],
       [ 0.95547078,  2.03234418,  0.99383009, -0.86593193, -0.06227458,
        -0.3018414 ],
       [-1.04660448, -1.38154001, -1.43596343,  1.15482518, -0.02036266,
        -0.3018414 ],
       [ 0.95547078,  0.74209164, -0.51778431,  1.15482518,  0.02995959,
        -0.3018414 ]])

In [ ]:
y_test[:5]

array([0., 0., 0., 0., 0.])

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `Binary Cross Entropy Loss` from `PyTorch`

In [ ]:
# def eval_genomes(genomes, config):
#     loss = nn.BCELoss()
#     loss = loss.to(device)
#     for genome_id, genome in genomes:
#         net = neat.nn.FeedForwardNetwork.create(genome, config)
#         preds = []
#         for xi in X_validate:
#             preds.append(1. if net.activate(xi)[0] > 0.5 else 0.)
#         correct = 0
#         for pred, truth in zip(preds, y_validate):
#             if pred == truth:
#                 correct += 1.
        
        
#         genome.fitness = float(correct / len(preds))
def eval_genomes(genomes, config):
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in X_validate:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds), torch.tensor(y_validate)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [ ]:
config_path = "./config-synthview"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


In [ ]:
base_config.pop_size

50

We also want to put a hard limit on how long this can go on for.

In [ ]:
maxNGenerations = 100

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [ ]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment: Vary dataset size

The first experiment is going to examine the difference from different dataset sizess

In [ ]:
datasetSizes = [
#         1000,
        2500,
#         5000,
#         10000,
#         25000,
#         50000,
        100000,
#         250000,
#         500000,
#         1000000,
#         1500000,
#         2000000
    ]
# datasetSizes = [1000]

In [ ]:
base_config.pop_size

50

In [ ]:
saveLocationTemplate = './../../data/experiments/synthview/experiment-dataset-{}-{}/'

In [ ]:
datasetLocation = './../../data/processed/synthetic_view/'
datasetFileTemplate = 'synthetic_view_test_{:07d}.csv'
# os.path.join(output_filepath, 'synthetic_view_test_{:07d}.csv'.format(dsSize)))

In [ ]:
X_train, X_validate, y_train, y_validate = load_dataset(os.path.join(datasetLocation, datasetFileTemplate.format(1000)))


In [ ]:
X_train[:5]

array([[ 0.96462528, -0.53797002, -0.9557672 ,  1.1560487 ,  0.48886043,
        -0.31831052],
       [-1.03667198,  1.76174535, -0.44512965, -0.86501546, -0.77320438,
         3.14158642],
       [-1.03667198,  0.08111723,  0.07925572, -0.86501546, -0.27909074,
        -0.31831052],
       [ 0.96462528, -0.03319542,  0.30570823,  1.1560487 , -0.8926339 ,
        -0.31831052],
       [ 0.96462528, -1.25741506, -1.46179745, -0.86501546, -1.27272262,
        -0.31831052]])

In [ ]:
y_train[:5]

array([1., 0., 0., 0., 0.])

## Start the experiment

In [ ]:
for dsSize in datasetSizes:
    for iteration_no in range(5):
        
        X_train, X_validate, y_train, y_validate = load_dataset(os.path.join(datasetLocation, datasetFileTemplate.format(dsSize)))
        X_train = torch.tensor(X_train)
        X_validate = torch.tensor(X_validate)
        y_train = torch.tensor(y_train).float()
        y_validate = torch.tensor(y_validate).float()
        
        
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting dsSize {} iteration {}".format(dsSize, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
        
        saveLocation = saveLocationTemplate.format(dsSize, iteration_no)
        
        p = instantiate_population(config, X_train, y_train, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations, nEpochs = 10)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(X_test, y_test):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
        
        
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished dsSize {} iteration {}".format(dsSize, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting dsSize 2500 iteration 0
Started at 07/26/2019, 03:09:22
################################################
################################################

 ****** Running generation 0 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(1.1663, grad_fn=<DivBackward0>)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Population's average fitness: 0.25354 stdev: 0.16072
Best fitness: 0.85022 - size: (1, 6) - species 2 - id 15
ending generation %s
Average adjusted fitness: 0.246
Mean genetic distance 3.658, standard deviation 1.649
Population of 50 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0     5      0.1    0.025     0
     2    0    17      0.9    0.380     0
     3    0    20      0.5    0.147     0
     4    0     8      0.8    0.433     0
Total extinctions: 0
Generation time: 237.349 sec

 ****** Running generation 1 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.5942, grad_fn=<DivBackward0>)
Population's average fitness: 0.50106 stdev: 0.28836
Best fitness: 1.61232 - size: (2, 7) - species 3 - id 70


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7f7e5f8549e8>, 'fitness': 1.6123156547546387, 'firstDerivatives': [0.0, 0.7620967626

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.2804, grad_fn=<DivBackward0>)
Population's average fitness: 1.65725 stdev: 0.90842
Best fitness: 3.28611 - size: (4, 6) - species 3 - id 302
ending generation %s
Average adjusted fitness: 0.388
Mean genetic distance 2.715, standard deviation 0.971
Population of 49 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    6     4      1.0    0.071     1
     2    6    12      2.1    0.437     0
     3    6    20      3.3    0.632     0
     4    6    13      1.7    0.410     0
Total extinctions: 0
Generation time: 343.043 sec (286.813 average)
Saving checkpoint to ./../../data/experiments/synthview/experiment-dataset-2500-0/checkpoint-6

 ****** Running generation 7 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.2401, grad_fn=<DivBackward0>)
Population's average fitness: 1.86382 s

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.2275, grad_fn=<DivBackward0>)
Population's average fitness: 2.26174 stdev: 1.30667
Best fitness: 4.13042 - size: (5, 9) - species 3 - id 413
ending generation %s
Average adjusted fitness: 0.425
Mean genetic distance 2.687, standard deviation 0.889
Population of 50 members in 4 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   10     9      1.8    0.293     0
     2   10    14      3.5    0.492     0
     3   10    19      4.1    0.678     1
     4   10     8      1.8    0.236     0
Total extinctions: 0
Generation time: 358.436 sec (317.697 average)
Saving checkpoint to ./../../data/experiments/synthview/experiment-dataset-2500-0/checkpoint-10

 ****** Running generation 11 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.2265, grad_fn=<DivBackward0>)
Population's average fitness: 2.47420